<a href="https://colab.research.google.com/github/Chpppy/Colab/blob/main/notebooks/LunarLander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gymnasium
!pip install stable_baselines3
!pip install sb3-contrib
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.1 MB/s eta 0:00:00


In [2]:
!pip install swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379449 sha256=6d7aec7897bbd5d3917c88bc468461e4d737468414eff24838f571bda1031e72
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


In [3]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [37]:
# config
N_TRIALS = 100  # Maximum number of trials
N_JOBS = 1 # Number of jobs to run in parallel
N_STARTUP_TRIALS = 5  # Stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 2  # Number of evaluations during the training
N_TIMESTEPS = int(2e4)  # Training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 60)  # 15 minutes

ENV_ID = "LunarLander-v3"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "env": ENV_ID,
}

In [21]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

In [38]:
from typing import Any, Dict
import torch
import torch.nn as nn

def sample_ppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for PPO hyperparameters.

    :param trial: Optuna trial object
    :return: The sampled hyperparameters for the given trial.
    """
    # Discount factor between 0.9 and 0.9999
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    # 8, 16, 32, ... 1024
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 9, 10)

    ### YOUR CODE HERE
    # TODO:
    # - define the learning rate search space [1e-5, 1] (log) -> `suggest_float`
    # - define the network architecture search space ["tiny", "small"] -> `suggest_categorical`
    # - define the activation function search space ["tanh", "relu"]
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1, log=True)
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    ### END OF YOUR CODE

    # Display true values
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("n_steps", n_steps)

    if net_arch == "tiny":
        net_arch = {"pi": [64], "vf": [64]}
    else:
        net_arch = {"pi": [64, 64], "vf": [64, 64]}


    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
        },
    }

In [39]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environement
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq:   Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

In [40]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function using by Optuna to evaluate
    one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters,
    evaluate it and report the result (mean episodic reward after training)

    :param trial: Optuna trial object
    :return: Mean episodic reward after training
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    ### YOUR CODE HERE
    # TODO:
    # 1. Sample hyperparameters and update the default keyword arguments: `kwargs.update(other_params)`
    # 2. Create the evaluation envs
    # 3. Create the `TrialEvalCallback`

    # 1. Sample hyperparameters and update the keyword arguments
    kwargs.update(sample_ppo_params(trial))
    # Create the RL model
    model = PPO(**kwargs, device="cpu")

    # 2. Create envs used for evaluation using `make_vec_env`, `ENV_ID` and `N_EVAL_ENVS`
    env = make_vec_env(env_id=ENV_ID, n_envs=N_EVAL_ENVS)
    # 3. Create the `TrialEvalCallback` callback defined above that will periodically evaluate
    # and report the performance using `N_EVAL_EPISODES` every `EVAL_FREQ`
    # TrialEvalCallback signature:
    # TrialEvalCallback(eval_env, trial, n_eval_episodes, eval_freq, deterministic, verbose)
    eval_callback = TrialEvalCallback(env, trial, N_EVAL_EPISODES, EVAL_FREQ, True, 1)

    ### END OF YOUR CODE

    nan_encountered = False
    try:
        # Train the model
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

In [41]:
import torch as th

# Set pytorch num threads to 1 for faster training
th.set_num_threads(1)
# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv("study_results_ppo_cartpole.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

[I 2025-02-24 08:54:09,772] A new study created in memory with name: no-name-5756be1a-f8bd-444b-a78f-3d70f9f244a2


Eval num_timesteps=10000, episode_reward=-233.81 +/- 114.16
Episode length: 136.40 +/- 23.50
New best mean reward!
Eval num_timesteps=20000, episode_reward=-164.40 +/- 53.72
Episode length: 687.80 +/- 173.60
New best mean reward!


[I 2025-02-24 08:55:11,449] Trial 0 finished with value: -164.3962309 and parameters: {'gamma': 0.000744730179195844, 'max_grad_norm': 2.6637881242995936, 'exponent_n_steps': 10, 'learning_rate': 0.001371402498309818, 'net_arch': 'small', 'activation_fn': 'tanh'}. Best is trial 0 with value: -164.3962309.


Eval num_timesteps=10000, episode_reward=-573.04 +/- 145.36
Episode length: 69.10 +/- 8.61
New best mean reward!
Eval num_timesteps=20000, episode_reward=-620.98 +/- 189.79
Episode length: 73.00 +/- 12.82


[I 2025-02-24 08:55:47,626] Trial 1 finished with value: -620.9750089 and parameters: {'gamma': 0.013998957413102604, 'max_grad_norm': 0.38078976749173177, 'exponent_n_steps': 9, 'learning_rate': 0.5689997431746266, 'net_arch': 'tiny', 'activation_fn': 'tanh'}. Best is trial 0 with value: -164.3962309.


...

Eval num_timesteps=10000, episode_reward=-231.32 +/- 69.13
Episode length: 1000.00 +/- 0.00
New best mean reward!


[I 2025-02-24 09:52:05,442] Trial 98 pruned. 


Eval num_timesteps=10000, episode_reward=-453.63 +/- 101.27
Episode length: 937.60 +/- 130.08
New best mean reward!
Eval num_timesteps=10000, episode_reward=-55.76 +/- 100.98
Episode length: 927.30 +/- 148.23
New best mean reward!
Eval num_timesteps=20000, episode_reward=-153.97 +/- 39.95
Episode length: 258.70 +/- 106.76


[I 2025-02-24 09:53:06,894] Trial 99 finished with value: -153.9720122 and parameters: {'gamma': 0.0003490546304377089, 'max_grad_norm': 1.223917011760367, 'exponent_n_steps': 10, 'learning_rate': 0.0006083387429401111, 'net_arch': 'small', 'activation_fn': 'tanh'}. Best is trial 63 with value: 171.7401334.


Number of finished trials:  100
Best trial:
  Value: 171.7401334
  Params: 
    gamma: 0.00017308880670360112
    max_grad_norm: 0.35278706193345905
    exponent_n_steps: 9
    learning_rate: 0.0029920996171534323
    net_arch: small
    activation_fn: relu
  User attrs:
    gamma_: 0.9998269111932964
    n_steps: 512


In [ ]:

#env = gym.make("LunarLander-v3")
policy_kwargs={
            "net_arch": {"pi": [64, 64], "vf": [64, 64]},
            "activation_fn": nn.ReLU,
        }
env = make_vec_env("LunarLander-v3", n_envs=5)
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=512,
    batch_size=64,
    n_epochs=4,
    gamma=0.9998269111932964,
    max_grad_norm= 0.35278706193345905,
    policy_kwargs=policy_kwargs,
    learning_rate= 0.0029920996171534323
)

model.learn(total_timesteps=300000)
# Save the model
model_name = "ppo-LunarLander-v3"
model.save(model_name)

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO

env = gym.make("LunarLander-v3", render_mode="human")
model = PPO.load("results/LunarLander/ppo-LunarLander-v3.zip")

obs, info = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()